In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import cv2
from PIL import Image
import numpy as np
import pandas as pd
import os
import sys
from sklearn.model_selection import train_test_split

In [2]:
mask_folder = os.path.join('AUG-Train', 'Train-Mask')
nomask_folder = os.path.join('AUG-Train', 'Train-NoMask')

In [3]:
maskpath, nomaskpath = [], []
for root, dirs, files in os.walk("AUG-Train"):
    for file in files:
        if file[:4] == 'mask':
            maskpath.append([os.path.join(root, file), 1])
        elif file[:2] == 'no':
            nomaskpath.append([os.path.join(root, file), 0])

In [4]:
data = []
image_size = 100
for i in nomaskpath + maskpath:
    img_array = cv2.imread(i[0])   # Uses BGR instead of RGB, have to change it back
    img_array = cv2.cvtColor(img_array, cv2.COLOR_BGR2RGB) # Converts to correct image encoding
    new_img_array = cv2.resize(img_array, (image_size, image_size))      # resizing the image array
    
    # encoding the labels. with_mask = 1 and without_mask = 0
    if i[1] == 1:
        data.append([new_img_array, 1])
    else:
        data.append([new_img_array, 0])

In [5]:
x = []
y = []
for image in data:
    x.append(image[0])
    y.append(image[1])

# converting x & y to numpy array as they are list
x = np.array(x)
y = np.array(y)

In [6]:
x = x/255

In [7]:
X_train, X_test, y_train, y_test = train_test_split(x, y)

In [8]:
# Defining the model
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=X_train[0].shape),      # flattening the image
    tf.keras.layers.Dense(1000, activation='relu'),             # start first layer with relu
    tf.keras.layers.Dense(20000, activation='swish'),           # then use swish
    tf.keras.layers.Dense(500, activation='swish'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])


lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-2,
    decay_steps=10000,
    decay_rate=0.9)

optimizer = tf.keras.optimizers.SGD(learning_rate=lr_schedule) # adam does not seem to work well with scheduling lr

model.compile(optimizer=optimizer,
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(X_train, y_train, epochs=200, batch_size = int(0.08 * len(X_train)))

Epoch 1/200
13/13 [==============================] - 6s 457ms/step - loss: 0.6654 - accuracy: 0.5618
Epoch 2/200
13/13 [==============================] - 6s 475ms/step - loss: 0.6212 - accuracy: 0.6385
Epoch 3/200
13/13 [==============================] - 6s 470ms/step - loss: 0.5746 - accuracy: 0.7078
Epoch 4/200
13/13 [==============================] - 6s 465ms/step - loss: 0.5222 - accuracy: 0.7694
Epoch 5/200
13/13 [==============================] - 6s 495ms/step - loss: 0.4724 - accuracy: 0.7989
Epoch 6/200
13/13 [==============================] - 6s 475ms/step - loss: 0.5269 - accuracy: 0.7347
Epoch 7/200
13/13 [==============================] - 6s 479ms/step - loss: 0.4728 - accuracy: 0.7659
Epoch 8/200
13/13 [==============================] - 6s 466ms/step - loss: 0.4406 - accuracy: 0.8192
Epoch 9/200
13/13 [==============================] - 6s 478ms/step - loss: 0.4655 - accuracy: 0.7863
Epoch 10/200
13/13 [==============================] - 6s 480ms/step - loss: 0.3475 - accura

13/13 [==============================] - 8s 604ms/step - loss: 0.1365 - accuracy: 0.9493
Epoch 82/200
13/13 [==============================] - 8s 596ms/step - loss: 0.1177 - accuracy: 0.9528
Epoch 83/200
13/13 [==============================] - 8s 597ms/step - loss: 0.1156 - accuracy: 0.9554
Epoch 84/200
13/13 [==============================] - 8s 604ms/step - loss: 0.1073 - accuracy: 0.9627
Epoch 85/200
13/13 [==============================] - 7s 572ms/step - loss: 0.4609 - accuracy: 0.8552
Epoch 86/200
13/13 [==============================] - 7s 542ms/step - loss: 0.1858 - accuracy: 0.9532
Epoch 87/200
13/13 [==============================] - 7s 551ms/step - loss: 0.1252 - accuracy: 0.9567
Epoch 88/200
13/13 [==============================] - 7s 561ms/step - loss: 0.1104 - accuracy: 0.9558
Epoch 89/200
13/13 [==============================] - 8s 588ms/step - loss: 0.1102 - accuracy: 0.9597
Epoch 90/200
13/13 [==============================] - 9s 723ms/step - loss: 0.0972 - accuracy: 

13/13 [==============================] - 6s 493ms/step - loss: 0.0680 - accuracy: 0.9775
Epoch 161/200
13/13 [==============================] - 6s 498ms/step - loss: 0.1019 - accuracy: 0.9588
Epoch 162/200
13/13 [==============================] - 7s 506ms/step - loss: 0.4143 - accuracy: 0.8743
Epoch 163/200
13/13 [==============================] - 6s 499ms/step - loss: 0.0879 - accuracy: 0.9705
Epoch 164/200
13/13 [==============================] - 6s 495ms/step - loss: 0.0679 - accuracy: 0.9783
Epoch 165/200
13/13 [==============================] - 6s 497ms/step - loss: 0.0595 - accuracy: 0.9835
Epoch 166/200
13/13 [==============================] - 7s 508ms/step - loss: 0.0553 - accuracy: 0.9848
Epoch 167/200
13/13 [==============================] - 8s 586ms/step - loss: 0.0496 - accuracy: 0.9866
Epoch 168/200
13/13 [==============================] - 7s 557ms/step - loss: 0.0490 - accuracy: 0.9853
Epoch 169/200
13/13 [==============================] - 8s 647ms/step - loss: 0.0462 - a

In [9]:
model.evaluate(X_test, y_test)

25/25 [==============================] - 2s 86ms/step - loss: 0.1260 - accuracy: 0.9429


[0.12601351737976074, 0.9428571462631226]

In [10]:
len(X_train)

2307

## Bring in test dataset

In [11]:
test_mask_path, test_nomask_path = [], []
for root, dirs, files in os.walk(os.path.join('test', 'with_mask')):
    for file in files:
        test_mask_path.append([os.path.join('test', 'with_mask',  file), 1])
for root, dirs, files in os.walk(os.path.join('test', 'without_mask')):
    for file in files:
        test_nomask_path.append([os.path.join('test', 'without_mask',  file),0])

In [12]:
testdata = []
image_size = 100
for i in test_mask_path+test_nomask_path:
    img_array = cv2.imread(i[0])   # Uses BGR instead of RGB, have to change it back
    img_array = cv2.cvtColor(img_array, cv2.COLOR_BGR2RGB) # Converts to correct image encoding
    new_img_array = cv2.resize(img_array, (image_size, image_size))      # resizing the image array
    
    # encoding the labels. with_mask = 1 and without_mask = 0
    if i[1] == 1:
        testdata.append([new_img_array, 1])
    else:
        testdata.append([new_img_array, 0])

In [13]:
x_test = []
y_test = []
for image in testdata:
    x_test.append(image[0])
    y_test.append(image[1])

# converting x & y to numpy array as they are list
x_test = np.array(x_test)
y_test = np.array(y_test)

In [14]:
x_test = x_test / 255

In [15]:
model.evaluate(x_test, y_test)

7/7 [==============================] - 0s 54ms/step - loss: 0.1869 - accuracy: 0.9192


[0.1869012415409088, 0.9191918969154358]

In [47]:
w = []
for i in test_mask_path+test_nomask_path:
    w.append(cv2.imread(i[0]).shape)

In [50]:
max(w)

(960, 960, 3)